### Friday, January 26, 2024

Going to try to get this to run but instead of hitting OpenAI, use LMStudio.

I am skipping the HuggingFaceHub section and working in the OpenAI section.

### Wednesday, November 22, 2023

Running again, along with the corresponding video.

[Getting Started with GPT-3 vs. Open Source LLMs - LangChain #1](https://www.youtube.com/watch?v=nE2skSRWTTs&list=PLIUOU7oqGTLieV9uTIFMm6_4PXg-hlN6F&index=1)

Start : OpenAI Usage => $1.63

End   : OpenAI Usage => $1.63


### Monday, November 20, 2023

https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/00-langchain-intro.ipynb


In [1]:
!ls /home/rob/Data2/huggingface/transformers

models--meta-llama--Llama-2-13b-chat-hf
models--meta-llama--Llama-2-13b-hf
models--meta-llama--Llama-2-7b-chat-hf
models--meta-llama--Llama-2-7b-hf
models--mistralai--Mistral-7B-Instruct-v0.1
models--mistralai--Mistral-7B-v0.1
tmpjiz9wrho
version.txt


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/00-langchain-intro.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/00-langchain-intro.ipynb)

#### [LangChain Handbook](https://github.com/pinecone-io/examples/tree/master/generation/langchain/handbook)

# Intro to LangChain

LangChain is a popular framework that allow users to quickly build apps and pipelines around **L**arge **L**anguage **M**odels. It can be used to for chatbots, **G**enerative **Q**uestion-**A**nwering (GQA), summarization, and much more.

The core idea of the library is that we can _"chain"_ together different components to create more advanced use-cases around LLMs. Chains may consist of multiple components from several modules:

* **Prompt templates**: Prompt templates are, well, templates for different types of prompts. Like "chatbot" style templates, ELI5 question-answering, etc

* **LLMs**: Large language models like GPT-3, BLOOM, etc

* **Agents**: Agents use LLMs to decide what actions should be taken, tools like web search or calculators can be used, and all packaged into logical loop of operations.

* **Memory**: Short-term memory, long-term memory.

In [2]:
# !pip install -qU langchain

# Using LLMs in LangChain

LangChain supports several LLM providers, like Hugging Face and OpenAI.

Let's start our exploration of LangChain by learning how to use a few of these different LLM integrations.

## Hugging Face

We first need to install additional prerequisite libraries:

In [3]:
# !pip install -qU huggingface_hub

For Hugging Face models we need a Hugging Face Hub API token. We can find this by first getting an account at [HuggingFace.co](https://huggingface.co/) and clicking on our profile in the top-right corner > click *Settings* > click *Access Tokens* > click *New Token* > set *Role* to *write* > *Generate* > copy and paste the token below:

In [2]:
import os
from getpass import getpass

os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass("HuggingFace API Key: ")

In [4]:
# import os
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'asdf'

We can then generate text using a HF Hub model (we'll use `google/flan-t5-x1`) using the Inference API built into Hugging Face Hub.

_(The default Inference API doesn't use specialized hardware and so can be slow and cannot run larger models like `bigscience/bloom-560m` or `google/flan-t5-xxl`)_

In [3]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

# initialize HF LLM
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":1e-10}
)

# build prompt template for simple question-answering
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(
    prompt=prompt,
    llm=flan_t5
)

question = "Which NFL team won the Super Bowl in the 2010 season?"

# print(llm_chain.run(question))

/usr/local/lib/python3.8/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [6]:
%%time

# This will probably time out ... 
print(llm_chain.run(question))

# 5m 34.3s ... timed out!

ValueError: Error raised by inference API: Model google/flan-t5-xl time out

If we'd like to ask multiple questions we can by passing a list of dictionary objects, where the dictionaries must contain the input variable set in our prompt template (`"question"`) that is mapped to the question we'd like to ask.

In [5]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
res = llm_chain.generate(qs)
res

# 5m 33.5s ... same time out period! Interesting ... !

ValueError: Error raised by inference API: Model google/flan-t5-xl time out

It is a LLM, so we can try feeding in all questions at once:

In [4]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=flan_t5
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

# print(llm_chain.run(qs_str))

In [8]:
print(llm_chain.run(qs_str))

# 5m 33.7s ... time out!

ValueError: Error raised by inference API: Model google/flan-t5-xl time out

But with this model it doesn't work too well, we'll see this approach works better with different models soon.

## OpenAI

Start by installing additional prerequisites:

In [9]:
# !pip install -qU openai

We can also use OpenAI's generative models. The process is similar, we need to
give our API key which can be retrieved by signing up for an account on the
[OpenAI website](https://openai.com/api/) (see top-right of page). We then pass the API key below:

In [7]:
import os

# os.environ['OPENAI_API_KEY'] = 'sk-asdf'

from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass("OpenAI API Key: ")

If using OpenAI via Azure you should also set:

```python
os.environ['OPENAI_API_TYPE'] = 'azure'
# API version to use (Azure has several)
os.environ['OPENAI_API_VERSION'] = '2022-12-01'
# base URL for your Azure OpenAI resource
os.environ['OPENAI_API_BASE'] = 'https://your-resource-name.openai.azure.com'
```

Then we decide on which model we'd like to use, there are several options but we will go with `text-davinci-003`:

In [8]:
# from langchain.llms import OpenAI

# davinci = OpenAI(model_name='text-davinci-003')

In [1]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

from langchain.llms import OpenAI

In [2]:
# Copied from some earlier cells ...

question = "Which NFL team won the Super Bowl in the 2010 season?"

# build prompt template for simple question-answering
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

In [3]:
# davinci = OpenAI(model_name='text-davinci-003')

davinci = OpenAI(base_url="http://localhost:1234/v1", api_key="NULL")

/home/rob/miniforge3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


Alternatively if using Azure OpenAI we do:

```python
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    deployment_name="your-azure-deployment", 
    model_name="text-davinci-003"
)
```

We'll use the same simple question-answer prompt template as before with the Hugging Face example. The only change is that we now pass our OpenAI LLM `davinci`:

In [4]:
llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

print(llm_chain.run(question))

# 0.8s

/home/rob/miniforge3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The New Orleans Saints won the Super Bowl XLIV in the 2010 NFL season. They defeated the Indianapolis Colts with a score of 31-28. Drew Brees was named Super Bowl MVP after completing 32 of 44 passes for 288 yards and two touchdowns. The Saints' defense also played a key role in the victory, forcing four turnovers and recording three sacks.


The same works again for multiple questions using `generate`:

In [5]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
llm_chain.generate(qs)

# 1.0s 

LLMResult(generations=[[Generation(text='There is no NFL team that won the Super Bowl in the 2010 season. The New Orleans Saints won Super Bowl XLIV in February 2010, but the 2010 season technically refers to the regular season and playoffs leading up to the Super Bowl.\n\nTo convert 6 feet 4 inches to centimeters, you can use the following conversion factor: 1 inch is equal to 2.54 centimeters. Therefore, 6 feet 4 inches is equal to approximately 198.12 centimeters (6 x 12 inches x 2.54 cm + 4 inches x 2.54 cm).\n\nThere have been only 12 people who have walked on the moon as of now, and they are all astronauts from the United States. The last person to walk on the moon was Buzz Aldrin in December 1972 during the Apollo 17 mission. Therefore, no NFL player has ever been to the moon.\n\nBlades of grass do not have eyes. They are simple plants that absorb nutrients and water from the soil through their roots and convert them into', generation_info={'finish_reason': 'stop', 'logprobs': N

Note that the below format doesn't feed the questions in iteratively but instead all in one chunk.

In [6]:
qs = [
    "Which NFL team won the Super Bowl in the 2010 season?",
    "If I am 6 ft 4 inches, how tall am I in centimeters?",
    "Who was the 12th person on the moon?",
    "How many eyes does a blade of grass have?"
]
print(llm_chain.run(qs))

The Pittsburgh Steelers won the Super Bowl in the 2010 NFL season. They defeated the Green Bay Packers with a score of 25-21 in Super Bowl XLV.

If you are 6 ft 4 inches tall, then you are approximately 193.04 centimeters tall. To calculate this conversion, there are 2.54 centimeters in one inch, so multiply the number of inches by 2.54 to get the height in centimeters.

No person other than the twelve astronauts who have walked on the moon can definitively answer this question. The twelfth person to walk on the moon was Buzz Aldrin, who did so as part of the Apollo 11 mission on July 20, 1969.

A blade of grass does not have eyes. It is a simple plant that doesn't possess any organs like eyes or other sensory structures.


Now we can try to answer all question in one go, as mentioned, more powerful LLMs like `text-davinci-003` will be more likely to handle these more complex queries.

In [7]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=davinci
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

# 1.6s

The NFL team that won the Super Bowl in the 2010 season was the New Orleans Saints. They defeated the Indianapolis Colts with a score of 31-28.

To convert height from feet and inches to centimeters, you first convert the inches to centimeters by multiplying by 2.54 (the number of centimeters in an inch). Then add the feet multiplied by 12 (the number of inches in a foot) times 2.54 to get the total height in centimeters. So, for your height of 6 ft 4 in, the calculation would be:
(6 ft) x 12 in/ft x 2.54 cm/in + (4 in) x 2.54 cm/in = 193.04 cm or approximately 193.1 cm when rounded to the nearest whole number.

There is no 12th person who walked on the moon. The twelve individuals who have walked on the moon are all astronauts from the United States: Neil Armstrong, Buzz Aldrin, Edwin "Buzz" Aldrin, Michael Collins,


---